In [1]:
import pandas as pd
import time
import os
import sys
sys.path.append(os.path.abspath('../..'))
print(sys.path)
from src.experiment.utils import calculate_oct
from src.experiment.approaches.tsp import run_tsp_encoding

['c:\\Users\\1st User\\Desktop\\GitHub\\product-ordering\\src\\evaluation', 'c:\\ProgramData\\Anaconda3\\python38.zip', 'c:\\ProgramData\\Anaconda3\\DLLs', 'c:\\ProgramData\\Anaconda3\\lib', 'c:\\ProgramData\\Anaconda3', '', 'c:\\ProgramData\\Anaconda3\\lib\\site-packages', 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\locket-0.2.1-py3.8.egg', 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32', 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib', 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin', 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\1st User\\.ipython', 'c:\\Users\\1st User\\Desktop\\GitHub\\product-ordering']


In [2]:
filename = 'C:/Users/1st User/Desktop/Master_Projekt/Produktionsplanung Linie 3 seit KW47_21.xlsx'

dict_dfs = pd.read_excel(filename, sheet_name=None)
print(list(dict_dfs.keys()))
dict_dfs[list(dict_dfs.keys())[0]]

['KW47', 'KW48', 'KW49', 'KW50', 'KW51', 'KW01', 'KW02', 'KW03', 'KW04', 'KW06', 'KW07', 'KW08', 'KW09', 'KW10', 'KW11', 'KW12', 'KW13', 'KW14', 'KW15', 'KW16', 'KW17']


,Linie,Produkt,Menge
0,3,23545,0.0
1,3,22276,60.0
2,3,22720,60.0
3,3,18919,75.0
4,3,15101,100.0
5,3,17151,50.0
6,3,15227,20.0
7,3,16214,50.0
8,3,15950,150.0
9,3,12020,160.0


In [3]:
filename = 'C:/Users/1st User/Desktop/Master_Projekt/plan_activity_L3.csv'

df_plan_activity = pd.read_csv(filename, sep=';', index_col=0)
df_plan_activity

,name,plan_activity
id,,
12020,Rivella Rot Tr 24x50 PE,19500
12021,Rivella Blau Tr 24x50 PE,19500
13327,Rivella Grüntee Tr 24x50 PE,19500
15097,Rivella Blau Sc 6x150 PE,13500
15099,Rivella Grüntee Sc 6x150 PE,13500
...,...,...
23542,Rivella Blau Gastro Promo Sc 8x150 PE,12500
23545,Sterilisation zu Wochenbeginn,1
23546,Zwischensterilisation,1


In [4]:
df_results = pd.DataFrame([], columns=['KW', 'IST-OCT', 'IST-GV', 'OPT-OCT', 'OPT-GV', 'time', 'numProducts'])

for kw, df_kw in dict_dfs.items():
    if kw in ['KW47', # Berechnungszeit zu hoch
              'KW03', # Zwischensterilisation zwischendurch
              'KW04', # Revision
              'KW08', # Revision
              'KW12', # Revision
              'KW16']: # Promoauftrag enthalten
        continue
    # if kw not in ['KW13']:
    #     continue

    print('KW:', kw)
    products_list = [str(p) for p in df_kw['Produkt'].to_list()]
    print(products_list)
    assert len(products_list) > 0

    # Clean up Zwischensterilisation
    while '23546' in products_list:
        assert '23546' == products_list[-1]
        products_list = products_list[:-1]

    occurences = {products_list[0]: 1}
    for i in range(1, len(products_list)):
        if products_list[i] == products_list[i - 1]:
            occurences[products_list[i]] += 1
        else:
            assert products_list[i] not in occurences
            occurences[products_list[i]] = 1

    assert products_list[0] == '23545'
    assert products_list[-1] == '23547'

    products = set(occurences.keys())

    t = time.time()
    _, opt_order, _ = run_tsp_encoding(products, run=0, start='23545', end='23547')
    print(opt_order)
    t = time.time() - t
    assert len(products) == len(opt_order)
    assert opt_order[0] == '23545'
    assert opt_order[-1] == '23547'
    ist_order = list(dict.fromkeys(products_list))
    ist_oct = calculate_oct(ist_order)
    opt_oct = calculate_oct(opt_order)

    overall_duration = 0
    for product in products_list:
        # print(product)
        quantity = float(df_kw[df_kw['Produkt'] == int(product)]['Menge'].fillna(0).iloc[0]) * 1000
        plan_activity = df_plan_activity.at[int(product), 'plan_activity']
        # print('quantity:', quantity)
        # print('plan_activity:', plan_activity)
        duration_product = quantity / plan_activity * 60
        # print('duration_product:', duration_product)
        overall_duration += duration_product
    # print('overall_duration:', overall_duration)
    ist_planned_stopps = ist_oct
    opt_planned_stopps = opt_oct
    ist_pa = (overall_duration - ist_planned_stopps) / overall_duration
    opt_pa = (overall_duration - opt_planned_stopps) / overall_duration

    result = [kw, ist_oct, ist_pa, opt_oct, opt_pa, t, len(products)]
    print(result)
    df_results.loc[len(df_results.index)] = result

df_results

KW: KW48
['23545', '15952', '13327', '15229', '15099', '15101', '22720', '22276', '15230', '22361', '19374', '22249', '16214', '15950', '15951', '12021', '23614', '21850', '23071', '21845', '21865', '21846', '23151', '23149', '23547', '23546']
['23545', '15229', '21845', '23071', '23614', '21850', '15950', '16214', '19374', '22361', '15230', '22720', '15101', '22276', '21846', '21865', '23149', '23151', '15952', '13327', '15099', '22249', '15951', '12021', '23547']
['KW48', 1920, 0.6714486806058777, 1890, 0.6765822949714109, 2.3796546459198, 25]
KW: KW49
['23545', '22362', '19375', '22721', '15097', '18920', '18919', '15101', '12020', '15950', '23152', '23155', '23154', '23357', '23358', '23547']
['23545', '22362', '19375', '22721', '18920', '15097', '15950', '12020', '18919', '15101', '23152', '23358', '23357', '23155', '23154', '23547']
['KW49', 1365, 0.7041437108887267, 1335, 0.7106460469131503, 0.09400296211242676, 16]
KW: KW50
['23545', '21849', '21968', '21845', '21847', '21846',

,KW,IST-OCT,IST-GV,OPT-OCT,OPT-GV,time,numProducts
0,KW48,1920,0.671449,1890,0.676582,2.379655,25
1,KW49,1365,0.704144,1335,0.710646,0.094003,16
2,KW50,1800,0.667558,1770,0.673098,32.052999,26
3,KW51,1320,0.691174,1305,0.694683,1.112200,20
4,KW01,1770,0.551128,1755,0.554932,0.807004,23
5,KW02,1140,0.778057,1125,0.780977,0.148000,15
6,KW06,1185,0.775363,1170,0.778207,0.134001,15
7,KW07,1635,0.722659,1605,0.727748,8.730003,23
8,KW09,1185,0.761798,1170,0.764813,0.181967,16
9,KW10,1275,0.769557,1260,0.772268,0.729001,18


In [6]:
df_results.to_csv('evaluation_results.csv')

In [5]:
ist = ['23545', '15951', '19375', '22275', '15231', '18920', '22721', '15099', '15229', '15952', '13327', '12020', '12020', '19374', '15230', '22720', '23547']
opt = ['23545', '19375', '12020', '12020', '15230', '22720', '19374', '13327', '15952', '15099', '15229', '15951', '15231', '22275', '22721', '18920', '23547']

df_ist_15 = pd.DataFrame([[i, ''] for i in ist], columns=['id', 'name'])
for _, row in df_ist_15.iterrows():
    for i in df_ist_15[df_ist_15['id'] == row['id']].index.to_list():
        df_ist_15.at[i, 'name'] = df_plan_activity.loc[int(row['id'])]['name']

df_ist_15.to_csv('df_ist_15.csv')

df_opt_15 = pd.DataFrame([[i, ''] for i in opt], columns=['id', 'name'])
for _, row in df_ist_15.iterrows():
    for i in df_opt_15[df_opt_15['id'] == row['id']].index.to_list():
        df_opt_15.at[i, 'name'] = df_plan_activity.loc[int(row['id'])]['name']

df_opt_15.to_csv('df_opt_15.csv')

In [6]:
ist = ['23545', '15951', '12021', '19375', '22362', '15231', '22721', '18920', '15097', '22275', '22720', '18919', '15101', '19374', '22361', '22249', '20745', '12020', '15950', '23547']
opt = ['23545', '15950', '12020', '20745', '18919', '22720', '15101', '22249', '19374', '22361', '15951', '12021', '22721', '15097', '18920', '22275', '15231', '22362', '19375', '23547']

df_ist_51 = pd.DataFrame([[i, ''] for i in ist], columns=['id', 'name'])
for _, row in df_ist_51.iterrows():
    for i in df_ist_51[df_ist_51['id'] == row['id']].index.to_list():
        df_ist_51.at[i, 'name'] = df_plan_activity.loc[int(row['id'])]['name']

df_ist_51.to_csv('df_ist_51.csv')

df_opt_51 = pd.DataFrame([[i, ''] for i in opt], columns=['id', 'name'])
for _, row in df_opt_51.iterrows():
    for i in df_opt_51[df_opt_51['id'] == row['id']].index.to_list():
        df_opt_51.at[i, 'name'] = df_plan_activity.loc[int(row['id'])]['name']

df_opt_51.to_csv('df_opt_51.csv')

In [10]:
print(calculate_oct(ist))
ist2 = ['23545', '22720', '18919', '15101', '19374', '22361', '22249', '20745', '12020', '15950', '15951', '12021', '19375', '22362', '15231', '22721', '18920', '15097', '22275', '23547']
opt2 = ['23545', '15951', '12021', '22721', '15097', '18920', '22275', '15231', '22362', '19375', '15950', '12020', '20745', '18919', '22720', '15101', '22249', '19374', '22361', '23547']
print(calculate_oct(ist2))
print(calculate_oct(opt2))
print(calculate_oct(opt))

1320
1305
1305
1305
